In [5]:
import pronto
import json
import urllib.request

fypo = pronto.Ontology('./fypo-base.obo')

starting_point  = fypo['FYPO:0000054'].subclasses().to_set()
data = []

for term in starting_point:
    value = {"id": int(term.id[5:]), "name": term.name, "desc": term.name}
    parents = term.superclasses(with_self=False, distance=1).to_set() & starting_point
    if parents:
        value['parent'] = int(parents.pop().id[5:])
    data.append(value)



# Let's use the Vega radial tree example as a basis of the visualization
view = json.load(urllib.request.urlopen("https://vega.github.io/vega/examples/tree-layout.vg.json"))

# First replace the default data with our own
view['data'][0].pop('url')
view['data'][0]['values'] = data
view['marks'][1]['encode']['enter']['tooltip'] = {"signal": "datum.desc"}
# view['signals'][4]['value'] = 'cluster'

# Render the clustered tree
display({"application/vnd.vega.v5+json": view}, raw=True)


In [31]:
import pronto
import json
import urllib.request

fypo = pronto.Ontology('./fypo-base.obo')

starting_point  = fypo['FYPO:0000054'].subclasses().to_set()
data = {'links': [], 'nodes': []}
indexes = {}

for i,term in enumerate(starting_point):
    indexes[term.id] = i

for term in starting_point:
    value = {"id": term.id, "name": term.name, "desc": term.name, "group":  8}
    data['nodes'].append(value)
    parents = term.superclasses(with_self=False, distance=1).to_set() & starting_point
    for parent in parents:
        link = {"source": indexes[term.id], "target": indexes[parent.id],"value":  1}
        data['links'].append(link)



# Let's use the Vega radial tree example as a basis of the visualization
view = json.load(urllib.request.urlopen("https://vega.github.io/vega/examples/force-directed-layout.vg.json"))

# First replace the default data with our own
view['data'][0].pop('url')
view['data'][0]['values'] = data
view['data'][1].pop('url')
view['data'][1]['values'] = data
view['marks'][0]['encode']['hover'] = {}
view['marks'][0]['encode']['hover']['tooltip'] = {"signal": '{"id":datum.id, "name":datum.name}'}
# view['signals'][4]['value'] = 'cluster'

# Render the clustered tree
display({"application/vnd.vega.v5+json": view}, raw=True)
